In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

def prepare_data(file_path):
    """Load and prepare the data"""
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    return df

def calculate_signals(df, short_window=20, long_window=50):
    """Calculate trading signals based on SMA crossover"""
    # Calculate SMAs
    df['SMA_short'] = df['close'].rolling(window=short_window).mean()
    df['SMA_long'] = df['close'].rolling(window=long_window).mean()
    
    # Generate signals
    df['signal'] = 0
    df.loc[df['SMA_short'] > df['SMA_long'], 'signal'] = 1
    df.loc[df['SMA_short'] < df['SMA_long'], 'signal'] = -1
    
    # Generate positions (1 = long, 0 = neutral)
    df['position'] = df['signal'].shift(1)
    df['position'].fillna(0, inplace=True)
    
    return df

def calculate_returns(df):
    """Calculate strategy returns and metrics"""
    # Calculate daily returns
    df['market_returns'] = df['close'].pct_change()
    df['strategy_returns'] = df['position'] * df['market_returns']
    
    # Calculate cumulative returns
    df['cumulative_market_returns'] = (1 + df['market_returns']).cumprod()
    df['cumulative_strategy_returns'] = (1 + df['strategy_returns']).cumprod()
    
    return df

def calculate_metrics(df):
    """Calculate performance metrics"""
    # Annual metrics (assuming 365 trading days)
    annual_market_return = df['market_returns'].mean() * 365
    annual_strategy_return = df['strategy_returns'].mean() * 365
    
    # Volatility
    market_volatility = df['market_returns'].std() * np.sqrt(365)
    strategy_volatility = df['strategy_returns'].std() * np.sqrt(365)
    
    # Sharpe Ratio (assuming risk-free rate of 2%)
    risk_free_rate = 0.02
    market_sharpe = (annual_market_return - risk_free_rate) / market_volatility
    strategy_sharpe = (annual_strategy_return - risk_free_rate) / strategy_volatility
    
    # Maximum Drawdown
    market_drawdown = (df['cumulative_market_returns'] / df['cumulative_market_returns'].cummax() - 1).min()
    strategy_drawdown = (df['cumulative_strategy_returns'] / df['cumulative_strategy_returns'].cummax() - 1).min()
    
    metrics = {
        'Annual Return': {'Market': annual_market_return, 'Strategy': annual_strategy_return},
        'Volatility': {'Market': market_volatility, 'Strategy': strategy_volatility},
        'Sharpe Ratio': {'Market': market_sharpe, 'Strategy': strategy_sharpe},
        'Max Drawdown': {'Market': market_drawdown, 'Strategy': strategy_drawdown}
    }
    
    return metrics

def plot_results(df):
    """Plot the results"""
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['cumulative_market_returns'], label='Market Returns', alpha=0.7)
    plt.plot(df.index, df['cumulative_strategy_returns'], label='Strategy Returns', alpha=0.7)
    plt.title('Market vs Strategy Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.show()

# Main execution
def run_strategy(file_path, short_window=20, long_window=50):
    """Run the complete strategy"""
    # Prepare data
    df = prepare_data(file_path)
    
    # Calculate signals and returns
    df = calculate_signals(df, short_window, long_window)
    df = calculate_returns(df)
    
    # Calculate metrics
    metrics = calculate_metrics(df)
    
    # Plot results
    plot_results(df)
    
    # Print metrics
    print("\nPerformance Metrics:")
    for metric, values in metrics.items():
        print(f"\n{metric}:")
        print(f"Market: {values['Market']:.4f}")
        print(f"Strategy: {values['Strategy']:.4f}")
    
    return df, metrics

# Run the strategy
df, metrics = run_strategy('BTC_2019_2023_1h.csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt

# Define a simple moving average crossover strategy
class SMACross(bt.Strategy):
    params = (
        ('fast', 10),  # Fast moving average period
        ('slow', 50),  # Slow moving average period
        ('risk_per_trade', 0.01),  # Risk per trade (1% of portfolio)
        ('leverage', 1),  # Leverage multiplier
        ('stop_loss_pct', 0.02),  # Stop loss as % of entry price
    )

    def __init__(self):
        # Initialize moving averages
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)

        self.order = None  # Track pending orders
        self.buy_price = None  # Entry price

    def next(self):
        if self.order:
            return

        # Entry logic: Buy on crossover of fast MA above slow MA
        if not self.position:  # Only enter a position if none exists
            if self.crossover > 0:
                size = (self.broker.getvalue() * self.params.risk_per_trade) / (self.data.close[0] * self.params.leverage)
                self.buy(size=size * self.params.leverage)
                self.buy_price = self.data.close[0]
        else:
            # Exit logic: Sell if fast MA crosses below slow MA or if stop-loss is hit
            if self.crossover < 0 or self.data.close[0] < self.buy_price * (1 - self.params.stop_loss_pct):
                self.sell(size=self.position.size)

# Backtesting function
def backtest_strategy(data_file, timeframe):
    # Load data
    data = pd.read_csv(data_file, parse_dates=['datetime'])
    data.set_index('datetime', inplace=True)

    # Convert data to Backtrader feed
    data_feed = bt.feeds.PandasData(dataname=data)

    # Initialize backtrader engine
    cerebro = bt.Cerebro()
    cerebro.adddata(data_feed)
    cerebro.addstrategy(SMACross)

    # Set broker settings
    cerebro.broker.set_cash(1000)  # Initial portfolio
    cerebro.broker.setcommission(commission=0.001)  # 0.1% trading fee

    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

    # Run backtest
    results = cerebro.run()
    strat = results[0]

    # Performance metrics
    sharpe_ratio = strat.analyzers.sharpe.get_analysis()['sharperatio']
    drawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
    trades = strat.analyzers.trades.get_analysis()
    total_trades = trades.total.total
    win_rate = trades.won.total / total_trades if total_trades > 0 else 0

    # Plot equity curve
    print(f"Sharpe Ratio: {sharpe_ratio}")
    print(f"Max Drawdown: {drawdown}%")
    print(f"Win Rate: {win_rate * 100}%")
    cerebro.plot()

# Test strategy on different timeframes
backtest_strategy('BTC_2019_2023_1d.csv', '1D')
backtest_strategy('BTC_2019_2023_1h.csv', '1H')
backtest_strategy('BTC_2019_2023_1m.csv', '1M')


In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Custom indicators
class ObvIndicator(bt.Indicator):
    """On-Balance Volume indicator implementation"""
    lines = ('obv',)
    params = ()
    
    def __init__(self):
        self.obv = 0
        
    def next(self):
        if self.data.close[0] > self.data.close[-1]:
            self.obv += self.data.volume[0]
        elif self.data.close[0] < self.data.close[-1]:
            self.obv -= self.data.volume[0]
        self.lines.obv[0] = self.obv

class MLPredictor(bt.Indicator):
    """Machine Learning predictor using Random Forest"""
    lines = ('prediction',)
    params = (('period', 20),)
    
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.trained = False
        
    def create_features(self):
        # Create features from price and volume data
        df = pd.DataFrame({
            'close': self.data.close.get(size=self.p.period),
            'volume': self.data.volume.get(size=self.p.period),
            'high': self.data.high.get(size=self.p.period),
            'low': self.data.low.get(size=self.p.period)
        })
        
        # Calculate technical features
        df['returns'] = df['close'].pct_change()
        df['vol_change'] = df['volume'].pct_change()
        df['price_range'] = (df['high'] - df['low']) / df['close']
        
        return df.fillna(0)
    
    def next(self):
        if len(self.data) > self.p.period:
            features = self.create_features()
            
            if not self.trained and len(features) >= self.p.period:
                # Create training labels (1 if price goes up, 0 if down)
                labels = (features['returns'].shift(-1) > 0).astype(int)
                features = features.drop(['returns'], axis=1)
                
                # Train the model
                X_train = features[:-1]
                y_train = labels[:-1]
                self.model.fit(X_train, y_train)
                self.trained = True
            
            if self.trained:
                # Make prediction
                latest_features = features.iloc[-1:].drop(['returns'], axis=1)
                self.lines.prediction[0] = self.model.predict(latest_features)[0]
            else:
                self.lines.prediction[0] = 0

class MultiTimeframeStrategy(bt.Strategy):
    """Multi-timeframe trading strategy with machine learning integration"""
    params = (
        ('ema_fast', 10),
        ('ema_slow', 50),
        ('rsi_period', 14),
        ('macd_fast', 12),
        ('macd_slow', 26),
        ('macd_signal', 9),
        ('atr_period', 14),
        ('bb_period', 20),
        ('bb_devfactor', 2),
        ('risk_perc', 0.02),
        ('trail_percent', 0.1),
    )
    
    def __init__(self):
        # Initialize indicators for primary timeframe
        self.ema_fast = bt.indicators.EMA(period=self.p.ema_fast)
        self.ema_slow = bt.indicators.EMA(period=self.p.ema_slow)
        self.rsi = bt.indicators.RSI(period=self.p.rsi_period)
        self.macd = bt.indicators.MACD(
            period_me1=self.p.macd_fast,
            period_me2=self.p.macd_slow,
            period_signal=self.p.macd_signal
        )
        self.atr = bt.indicators.ATR(period=self.p.atr_period)
        self.bbands = bt.indicators.BollingerBands(
            period=self.p.bb_period,
            devfactor=self.p.bb_devfactor
        )
        self.obv = ObvIndicator()
        self.ml_predictor = MLPredictor()
        
        # Initialize indicators for higher timeframes
        self.h1_data = self.datas[1]  # 1-hour data
        self.d1_data = self.datas[2]  # 1-day data
        
        self.h1_ema_fast = bt.indicators.EMA(self.h1_data, period=self.p.ema_fast)
        self.h1_ema_slow = bt.indicators.EMA(self.h1_data, period=self.p.ema_slow)
        
        self.d1_ema_fast = bt.indicators.EMA(self.d1_data, period=self.p.ema_fast)
        self.d1_ema_slow = bt.indicators.EMA(self.d1_data, period=self.p.ema_slow)
        
        # Track orders and stops
        self.order = None
        self.stop_order = None
        self.take_profit_order = None
        self.trail_stop_order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
            
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
                
        self.order = None
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
        
    def check_entry_conditions(self):
        # Check multiple timeframe trend alignment
        m1_trend = self.ema_fast[0] > self.ema_slow[0]
        h1_trend = self.h1_ema_fast[0] > self.h1_ema_slow[0]
        d1_trend = self.d1_ema_fast[0] > self.d1_ema_slow[0]
        
        # Check technical conditions
        rsi_oversold = self.rsi[0] < 30
        macd_cross = self.macd.macd[0] > self.macd.signal[0] and self.macd.macd[-1] <= self.macd.signal[-1]
        bb_bounce = self.data.close[0] <= self.bbands.lines.bot[0]
        
        # Check ML prediction
        ml_bullish = self.ml_predictor.lines.prediction[0] == 1
        
        # Volume confirmation
        volume_confirm = self.obv[0] > self.obv[-1]
        
        return (m1_trend and h1_trend and d1_trend and 
                (rsi_oversold or macd_cross or bb_bounce) and 
                ml_bullish and volume_confirm)
    
    def next(self):
        if self.order:
            return
            
        # Calculate position size based on risk
        risk_amount = self.broker.get_cash() * self.p.risk_perc
        atr_stops = self.atr[0] * 2
        position_size = risk_amount / atr_stops
        
        if not self.position:
            if self.check_entry_conditions():
                # Enter long position
                self.order = self.buy(size=position_size)
                
                # Set stop loss and take profit
                stop_price = self.data.close[0] - atr_stops
                take_profit = self.data.close[0] + (atr_stops * 2)
                
                self.stop_order = self.sell(size=position_size, exectype=bt.Order.Stop, price=stop_price)
                self.take_profit_order = self.sell(size=position_size, exectype=bt.Order.Limit, price=take_profit)
                
                # Set trailing stop
                trail_price = self.data.close[0] * (1 - self.p.trail_percent)
                self.trail_stop_order = self.sell(size=position_size, exectype=bt.Order.StopTrail, trailpercent=self.p.trail_percent)
        
        else:
            # Update trailing stop if necessary
            if self.trail_stop_order and self.trail_stop_order.alive():
                new_trail_price = self.data.close[0] * (1 - self.p.trail_percent)
                if new_trail_price > self.trail_stop_order.created.price:
                    self.trail_stop_order.cancel()
                    self.trail_stop_order = self.sell(size=self.position.size, exectype=bt.Order.StopTrail, trailpercent=self.p.trail_percent)

class TradingSystem:
    """Main trading system class for running the strategy"""
    def __init__(self, data_path, start_cash=1000.0):
        self.cerebro = bt.Cerebro()
        self.data_path = data_path
        self.start_cash = start_cash
        
    def load_data(self):
        # Load data for different timeframes
        data_m1 = bt.feeds.GenericCSVData(
            dataname=self.data_path,
            timeframe=bt.TimeFrame.Minutes
        )
        data_h1 = bt.feeds.GenericCSVData(
            dataname=self.data_path,
            timeframe=bt.TimeFrame.Minutes,
            compression=60
        )
        data_d1 = bt.feeds.GenericCSVData(
            dataname=self.data_path,
            timeframe=bt.TimeFrame.Minutes,
            compression=1440
        )
        
        self.cerebro.adddata(data_m1)
        self.cerebro.adddata(data_h1)
        self.cerebro.adddata(data_d1)
        
    def setup_backtest(self):
        # Set up the backtest parameters
        self.cerebro.addstrategy(MultiTimeframeStrategy)
        self.cerebro.broker.setcash(self.start_cash)
        self.cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
        self.cerebro.addsizer(bt.sizers.PercentSizer, percents=95)
        
        # Add analyzers
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
        self.cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
        self.cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
        self.cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
        
    def run(self):
        self.load_data()
        self.setup_backtest()
        results = self.cerebro.run()
        self.analyze_results(results[0])
        self.cerebro.plot(style='candlestick')
        
    def analyze_results(self, strategy):
        # Calculate and display performance metrics
        sharpe_ratio = strategy.analyzers.sharpe.get_analysis()['sharperatio']
        drawdown = strategy.analyzers.drawdown.get_analysis()['max']['drawdown']
        returns = strategy.analyzers.returns.get_analysis()['rtot']
        trades = strategy.analyzers.trades.get_analysis()
        
        win_rate = trades['won'] / (trades['won'] + trades['lost']) if (trades['won'] + trades['lost']) > 0 else 0
        
        print("\nPerformance Metrics:")
        print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
        print(f"Maximum Drawdown: {drawdown:.2%}")
        print(f"Total Return: {returns:.2%}")
        print(f"Win Rate: {win_rate:.2%}")

# Example usage
if __name__ == "__main__":
    trading_system = TradingSystem("BTC_2019_2023_1h.csv", start_cash=1000.0)
    trading_system.run()

test

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_obv(df):
    obv = [0]
    for i in range(1, len(df)):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            obv.append(obv[-1] + df['volume'].iloc[i])
        elif df['close'].iloc[i] < df['close'].iloc[i-1]:
            obv.append(obv[-1] - df['volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return pd.Series(obv, index=df.index)

def calculate_macd(df, fast=12, slow=26, signal=9):
    exp1 = df['close'].ewm(span=fast, adjust=False).mean()
    exp2 = df['close'].ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def calculate_rsi(df, periods=14):
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_stochastic(df, k_period=14, d_period=3):
    low_min = df['low'].rolling(window=k_period).min()
    high_max = df['high'].rolling(window=k_period).max()
    k = 100 * ((df['close'] - low_min) / (high_max - low_min))
    d = k.rolling(window=d_period).mean()
    return k, d

def implement_strategy(file_path):
    # Read data
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    
    # Calculate indicators
    df['OBV'] = calculate_obv(df)
    df['MACD'], df['Signal_Line'] = calculate_macd(df)
    df['RSI'] = calculate_rsi(df)
    df['Stoch_K'], df['Stoch_D'] = calculate_stochastic(df)
    
    # Initialize positions and portfolio
    position = 0
    portfolio = 1000
    trades = []
    entry_price = 0
    
    # Trading signals
    for i in range(len(df)):
        if i < 26:  # Skip initial periods due to indicator calculation
            continue
            
        # Entry conditions for long position
        long_condition = (
            df['MACD'].iloc[i] > df['Signal_Line'].iloc[i] and
            df['RSI'].iloc[i] > 30 and
            df['Stoch_K'].iloc[i] > df['Stoch_D'].iloc[i] and
            df['OBV'].iloc[i] > df['OBV'].iloc[i-1]
        )
        
        # Entry conditions for short position
        short_condition = (
            df['MACD'].iloc[i] < df['Signal_Line'].iloc[i] and
            df['RSI'].iloc[i] < 70 and
            df['Stoch_K'].iloc[i] < df['Stoch_D'].iloc[i] and
            df['OBV'].iloc[i] < df['OBV'].iloc[i-1]
        )
        
        # Execute trades
        if position == 0:
            if long_condition:
                position = 1
                entry_price = df['close'].iloc[i]
                portfolio *= (1 - 0.001)  # 0.1% fee
                trades.append(('LONG', df.index[i], entry_price, portfolio))
            elif short_condition:
                position = -1
                entry_price = df['close'].iloc[i]
                portfolio *= (1 - 0.001)
                trades.append(('SHORT', df.index[i], entry_price, portfolio))
                
        # Exit conditions
        elif position == 1 and short_condition:
            position = 0
            exit_price = df['close'].iloc[i]
            portfolio *= (1 - 0.001) * (exit_price / entry_price)
            trades.append(('EXIT_LONG', df.index[i], exit_price, portfolio))
            
        elif position == -1 and long_condition:
            position = 0
            exit_price = df['close'].iloc[i]
            portfolio *= (1 - 0.001) * (2 - exit_price / entry_price)
            trades.append(('EXIT_SHORT', df.index[i], exit_price, portfolio))
    
    # Calculate performance metrics
    trades_df = pd.DataFrame(trades, columns=['Type', 'DateTime', 'Price', 'Portfolio'])
    
    # Market returns
    market_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
    strategy_return = (portfolio / 1000 - 1) * 100
    
    # Calculate max drawdown
    peak = trades_df['Portfolio'].expanding(min_periods=1).max()
    drawdown = (trades_df['Portfolio'] - peak) / peak
    max_drawdown = drawdown.min() * 100
    
    # Calculate Sharpe ratio (assuming risk-free rate = 0)
    returns = pd.Series([t[3] for t in trades]).pct_change()
    sharpe_ratio = np.sqrt(252) * (returns.mean() / returns.std())
    
    # Calculate win rate
    profitable_trades = sum(trades_df['Portfolio'].diff() > 0)
    total_trades = len(trades_df)
    win_rate = (profitable_trades / total_trades) * 100 if total_trades > 0 else 0
    
    print(f"Performance Metrics:")
    print(f"Total Return: {strategy_return:.2f}% (Market: {market_return:.2f}%)")
    print(f"Max Drawdown: {max_drawdown:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Win Rate: {win_rate:.2f}%")
    print(f"Number of Trades: {total_trades}")
    
    return trades_df

if __name__ == "__main__":
    file_path = "BTC_2019_2023_1m.csv"
    trades_df = implement_strategy(file_path)

Performance Metrics:
Total Return: -100.00% (Market: 323.32%)
Max Drawdown: -100.00%
Sharpe Ratio: -6.68
Win Rate: 10.22%
Number of Trades: 287335


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

def calculate_bollinger_bands(df, window=20, num_sd=2):
    sma = df['close'].rolling(window).mean()
    std = df['close'].rolling(window).std()
    return sma + (std * num_sd), sma - (std * num_sd)

def calculate_macd(df, fast=12, slow=26, signal=9):
    exp1 = df['close'].ewm(span=fast, adjust=False).mean()
    exp2 = df['close'].ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def calculate_stochastic(df, k_period=14, d_period=3):
    low_min = df['low'].rolling(window=k_period).min()
    high_max = df['high'].rolling(window=k_period).max()
    k = 100 * ((df['close'] - low_min) / (high_max - low_min))
    d = k.rolling(window=d_period).mean()
    return k, d

def calculate_atr(df, window=14):
    tr1 = df['high'] - df['low']
    tr2 = abs(df['high'] - df['close'].shift(1))
    tr3 = abs(df['low'] - df['close'].shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window).mean()

def calculate_obv(df):
    obv = [0]
    for i in range(1, len(df)):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            obv.append(obv[-1] + df['volume'].iloc[i])
        elif df['close'].iloc[i] < df['close'].iloc[i-1]:
            obv.append(obv[-1] - df['volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return pd.Series(obv, index=df.index)

def hawkes_process(data: pd.Series, kappa: float = 3.0):
    alpha = np.exp(-kappa)
    arr = data.to_numpy()
    output = np.zeros(len(data))
    output[:] = np.nan
    
    for i in range(1, len(data)):
        if np.isnan(output[i - 1]):
            output[i] = arr[i]
        else:
            output[i] = output[i - 1] * alpha + arr[i]
    
    return pd.Series(output, index=data.index) * kappa

def implement_strategy(file_path):
    # Read data
    df = pd.read_csv(file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    
    # Calculate indicators
    df['BB_upper'], df['BB_lower'] = calculate_bollinger_bands(df)
    df['MACD'], df['Signal_Line'] = calculate_macd(df)
    df['Stoch_K'], df['Stoch_D'] = calculate_stochastic(df)
    df['ATR'] = calculate_atr(df)
    df['OBV'] = calculate_obv(df)
    
    # Calculate Hawkes process on volume
    df['vol_normalized'] = (df['volume'] - df['volume'].mean()) / df['volume'].std()
    df['volume_hawkes'] = hawkes_process(df['vol_normalized'])
    
    # Initialize positions and portfolio
    position = 0
    portfolio = 1000
    trades = []
    entry_price = 0
    
    # Trading signals
    for i in range(len(df)):
        if i < 26:  # Skip initial periods
            continue
            
        # Volume spike prediction from Hawkes process
        volume_spike = df['volume_hawkes'].iloc[i] > df['volume_hawkes'].rolling(20).mean().iloc[i]
        
        # Long entry conditions
        long_condition = (
            volume_spike and
            df['close'].iloc[i] <= df['BB_lower'].iloc[i] and
            df['MACD'].iloc[i] > df['Signal_Line'].iloc[i] and
            df['Stoch_K'].iloc[i] > 20 and df['Stoch_K'].iloc[i] < df['Stoch_D'].iloc[i] and
            df['OBV'].iloc[i] > df['OBV'].iloc[i-1] and
            df['ATR'].iloc[i] > df['ATR'].rolling(20).mean().iloc[i]
        )
        
        # Short entry conditions
        short_condition = (
            volume_spike and
            df['close'].iloc[i] >= df['BB_upper'].iloc[i] and
            df['MACD'].iloc[i] < df['Signal_Line'].iloc[i] and
            df['Stoch_K'].iloc[i] < 80 and df['Stoch_K'].iloc[i] > df['Stoch_D'].iloc[i] and
            df['OBV'].iloc[i] < df['OBV'].iloc[i-1] and
            df['ATR'].iloc[i] > df['ATR'].rolling(20).mean().iloc[i]
        )
        
        # Execute trades
        if position == 0:
            if long_condition:
                position = 1
                entry_price = df['close'].iloc[i]
                portfolio *= (1 - 0.001)
                trades.append(('LONG', df.index[i], entry_price, portfolio))
            elif short_condition:
                position = -1
                entry_price = df['close'].iloc[i]
                portfolio *= (1 - 0.001)
                trades.append(('SHORT', df.index[i], entry_price, portfolio))
                
        # Exit conditions
        elif position == 1 and short_condition:
            position = 0
            exit_price = df['close'].iloc[i]
            portfolio *= (1 - 0.001) * (exit_price / entry_price)
            trades.append(('EXIT_LONG', df.index[i], exit_price, portfolio))
            
        elif position == -1 and long_condition:
            position = 0
            exit_price = df['close'].iloc[i]
            portfolio *= (1 - 0.001) * (2 - exit_price / entry_price)
            trades.append(('EXIT_SHORT', df.index[i], exit_price, portfolio))
    
    # Calculate performance metrics
    trades_df = pd.DataFrame(trades, columns=['Type', 'DateTime', 'Price', 'Portfolio'])
    
    # Market returns
    market_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
    strategy_return = (portfolio / 1000 - 1) * 100
    
    # Max drawdown
    peak = trades_df['Portfolio'].expanding(min_periods=1).max()
    drawdown = (trades_df['Portfolio'] - peak) / peak
    max_drawdown = drawdown.min() * 100
    
    # Sharpe ratio
    returns = pd.Series([t[3] for t in trades]).pct_change()
    sharpe_ratio = np.sqrt(252) * (returns.mean() / returns.std())
    
    # Win rate
    profitable_trades = sum(trades_df['Portfolio'].diff() > 0)
    total_trades = len(trades_df)
    win_rate = (profitable_trades / total_trades) * 100 if total_trades > 0 else 0
    
    print(f"Performance Metrics:")
    print(f"Total Return: {strategy_return:.2f}% (Market: {market_return:.2f}%)")
    print(f"Max Drawdown: {max_drawdown:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Win Rate: {win_rate:.2f}%")
    print(f"Number of Trades: {total_trades}")
    
    return trades_df

if __name__ == "__main__":
    file_path = "BTC_2019_2023_1m.csv"
    trades_df = implement_strategy(file_path)

KeyboardInterrupt: 